In [1]:
import cv2
import imutils
from imutils import paths
import random
import numpy as np
import os
from scipy.cluster.vq import *
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import sklearn.metrics as metrics

In [9]:
imagePaths = list(imutils.paths.list_images("Bikes")) + list(imutils.paths.list_images("Horses"))
random.shuffle(imagePaths)

labels = []
descriptorList = []

sift = cv2.xfeatures2d.SIFT_create()

for (i, imagePath) in enumerate(imagePaths):
    image = cv2.imread(imagePath)
    gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    label = imagePath.split(os.path.sep)[0].split("/")[0]
    keyPoints, descriptor = sift.detectAndCompute(image, None)
    descriptorList.append((imagePath, descriptor))
    if (label == 'Bikes'):
        labels.append(0)
    else:
        labels.append(1)
        
descriptors = descriptorList[0][1]
    
for imageFlag, descriptor in descriptorList[1:]:
    descriptors = np.vstack((descriptors, descriptor))
    

Horses/h1.jpg
Bikes/0003.jpg
Horses/horse27.jpg
Bikes/0031.jpg
Horses/horse23.jpg
Bikes/0005.jpg
Bikes/0009.jpg
Bikes/0048.jpg
Horses/horse43.jpg
Bikes/0020.jpg
Horses/horse8 .jpg
Bikes/0027.jpg
Bikes/0068.jpg
Bikes/0045.jpg
Bikes/0029.jpg
Bikes/0065.jpg
Bikes/0034.jpg
Bikes/0042.jpg
Bikes/0056.jpg
Horses/horse80.jpg
Horses/horse6 .jpg
Horses/horse7 .jpg
Horses/horse45.jpg
Horses/horse3 .jpg
Horses/horse1  (6).jpg
Bikes/0054.jpg
Bikes/0052.jpg
Bikes/0038.jpg
Horses/horse1  (8).jpg
Horses/horse28.jpg
Bikes/0060.jpg
Horses/horse41.jpg
Horses/horse33.jpg
Horses/horse37.jpg
Horses/horse12.jpg
Bikes/0080.jpg
Horses/horse29.jpg
Horses/horse63.jpg
Bikes/0032.jpg
Bikes/0037.jpg
Bikes/0022.jpg
Bikes/0016.jpg
Horses/horse1 .jpg
Horses/h2.jpg
Bikes/0010.jpg
Horses/horse32.jpg
Horses/horse81.jpg
Bikes/0012.jpg
Bikes/0064.jpg
Horses/horse76.jpg
Horses/horse73.jpg
Horses/horse22.jpg
Horses/horse18.jpg
Bikes/0024.jpg
Horses/horse71.jpg
Horses/horse35.jpg
Bikes/0013.jpg
Bikes/0014.jpg
Bikes/0074.jpg
B

In [3]:
k = 500

voc,  variance = kmeans(descriptors, k, 1)
print (variance)
print (len(voc))

KeyboardInterrupt: 

In [ ]:
imageFeatures = np.zeros((len(imagePaths), k), "float32")
for i in range(len(imagePaths)):
    words, distance = vq(descriptorList[i][1],voc)
    for w in words:
        imageFeatures[i][w] += 1

In [ ]:
# numberOccurences = np.sum( (imageFeatures > 0) * 1, axis = 0)
# # print (len(numberOccurences))
# idf = np.array(np.log((1.0*len(imagePaths)+1) / (1.0*numberOccurences + 1)), 'float32')
# print (len(idf))

In [ ]:
stdSlr = StandardScaler().fit(imageFeatures)
imageFeatures = stdSlr.transform(imageFeatures)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(imageFeatures, labels, test_size=0.4, random_state=4)

In [ ]:
clf = cv2.ml.KNearest_create()
clf.train(X_train, cv2.ml.ROW_SAMPLE, np.asarray(y_train, dtype=np.float32))

In [ ]:
ret, results, neighbours ,dist = clf.findNearest(X_test, k=10)

In [ ]:
pred_label = []
for var in results:
    label = var
    pred_label.append(int(label))

print (y_test)
print (pred_label)
    
metrics.accuracy_score(y_test, pred_label)
# results = list(results.astype(int))
# print (results)
# print (labels_test)